In [23]:
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
import numpy as np
from datetime import datetime

In [24]:
def fit_all_models(par,one_year,three_year,SP_log,dates):
    """ Fits ARIMA models with the parameters specified in par
    to the provided data for every index. 
    
    Parameters: 
    par - Dictionary of parameters
    one_year - Series of one year rates
    three_year - Series of three year rates
    SP_log - Logarithmed S&P500-index data 
    dates - The dates to be fitted 
    
    Returns:
    A pandas dataframe containing three series with fitted models
    for every date in the data
    
    """
    ARIMA_models = pd.DataFrame()
    d = timedelta(days=par['lookback'])
    start_params_oyr = [0,0.5,0.5]
    start_params_tyr = [0,0.5,0.5]
    start_params_sp = [0,0.5,0.5]
    ctr = 0
    t = time.time()
    for date in dates: 
        if not ctr % 50: 
            print("Processing {}".format(date.strftime("%Y-%m-%d")))
        if date > dates[0] + timedelta(days=par['lookback']):
            m_oyr = ARIMA(one_year[:date.strftime('%Y-%m-%d')],order=(par['p'],par['d'],par['q']))
            m_oyr_fit = m_oyr.fit(start_params=start_params_oyr)
            m_tyr = ARIMA(three_year[:date.strftime('%Y-%m-%d')],order=(par['p'],par['d'],par['q']))
            m_tyr_fit = m_tyr.fit(start_params=start_params_tyr)
            m_sp = ARIMA(SP_log[:date.strftime('%Y-%m-%d')],order=(par['p'],par['d'],par['q']))
            m_sp_fit = m_sp.fit(start_params=start_params_sp)
            ARIMA_models = ARIMA_models.append({'date':date,
                                                '1 YEAR':m_oyr_fit,
                                                '3 YEAR':m_tyr_fit,
                                                'S&P':m_sp_fit
                                               },ignore_index=True) 
            start_params_oyr = m_oyr_fit.params.values
            start_params_tyr = m_tyr_fit.params.values
            start_params_sp = m_sp_fit.params.values
        ctr += 1
    ARIMA_models.set_index('date',inplace=True)
    total_time = round(time.time() - t,3)
    print("Total time: {} s".format(total_time))
    print("Average time per fitting: {} s".format(round(total_time/(3 * len(dates)),3)))
    return ARIMA_models

def predict_arima(ARIMA_models, steps):
    """ Predicts the values steps ahead from the models in ARIMA_models. 
    
    Parameters: 
    ARIMA_models - A dataframe with models named as implied above.
    steps - How many days ahead to predict. 
    
    Returns: 
    A dataframe containing the prediction steps ahead for each category. 
    The date is the date when the forecast is made and not the day we want to forecast. 
    """
    ARIMA_preds = pd.DataFrame({'date':[], '1 YEAR':[], '3 YEAR': [], 'S&P': []})
    for d in ARIMA_models.index:
        ARIMA_preds = ARIMA_preds.append({'date': d,
                                          '1 YEAR': ARIMA_models.loc[d]['1 YEAR'].forecast(steps=steps)[0][-1],
                                          '3 YEAR': ARIMA_models.loc[d]['3 YEAR'].forecast(steps=steps)[0][-1],
                                          'S&P': ARIMA_models.loc[d]['S&P'].forecast(steps=steps)[0][-1]},
                                        ignore_index=True)
    ARIMA_preds.set_index('date',inplace=True)  
    return ARIMA_preds